In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

In [29]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '/home/sawan/Desktop/minor_1'
width=256
height=256
depth=3

In [30]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [31]:
image_list, label_list = [], []
try:
    print(" Loading images ...")
    root_dir = listdir(directory_root)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for plant_disease_folder in plant_disease_folder_list:
            print(f"Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print(" Completed")  
except Exception as e:
    print(f"Error : {e}")

 Loading images ...
Processing Tomato__Tomato_mosaic_virus ...
Processing Tomato_Late_blight ...
Processing Tomato_Bacterial_spot ...
Processing Tomato_Leaf_Mold ...
Processing Tomato__Target_Spot ...
Processing Pepper__bell___Bacterial_spot ...
Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
Processing Tomato_Septoria_leaf_spot ...
Processing Potato___healthy ...
Processing Potato___Early_blight ...
Processing Potato___Late_blight ...
Processing Tomato_healthy ...
Processing Pepper__bell___healthy ...
Processing Tomato_Early_blight ...
Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
 Completed


In [32]:
image_size = len(image_list)
print(image_size)

2952


Transform Image Labels uisng [Scikit Learn](http://scikit-learn.org/)'s LabelBinarizer

In [33]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))


In [34]:
print(label_binarizer.classes_)

['Pepper__bell___Bacterial_spot' 'Pepper__bell___healthy'
 'Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy'
 'Tomato_Bacterial_spot' 'Tomato_Early_blight' 'Tomato_Late_blight'
 'Tomato_Leaf_Mold' 'Tomato_Septoria_leaf_spot'
 'Tomato_Spider_mites_Two_spotted_spider_mite' 'Tomato__Target_Spot'
 'Tomato__Tomato_YellowLeaf__Curl_Virus' 'Tomato__Tomato_mosaic_virus'
 'Tomato_healthy']


In [35]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [36]:
print(" Spliting images to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

 Spliting images to train, test


In [37]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [38]:
disease_type = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
disease_type.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape, activation = 'relu'))
disease_type.add(BatchNormalization(axis=chanDim))
disease_type.add(MaxPooling2D(pool_size=(2,2)))
disease_type.add(Dropout(0.3))


disease_type.add(Conv2D(64, (3, 3), padding="same",activation = 'relu'))
disease_type.add(BatchNormalization(axis=chanDim))
disease_type.add(MaxPooling2D(pool_size=(2, 2)))
disease_type.add(Dropout(0.3))

disease_type.add(Flatten())
disease_type.add(BatchNormalization())
disease_type.add(Dropout(0.3))
disease_type.add(Dense(activation="relu", units=128))
disease_type.add(Dense(activation="sigmoid", units=15))

In [39]:
disease_type.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])

In [40]:
history = disease_type.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

Epoch 1/1
73/73 [==============================] - 35s 478ms/step - loss: 1.1960 - acc: 0.9009 - val_loss: 1.3085 - val_acc: 0.9105


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

Model Accuracy

In [ ]:
print(" model accuracy")
scores = disease_type.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
# save the model to disk

pickle.dump(disease_type,open('cnn_model.pkl', 'wb'))